<h1 style="text-align:center;">Computational Intelligence</h1>
<h2 style="text-align:center;">Rule-based Fuzzy Control System</h2>
<h4 style="text-align:center;">by H. Naderan</h4>
<h5 style="text-align:center;">
Mechanical Engineering Department<br>
Amirkabir University of Technology
</h5>

***

### Description of the problem
We want to design a control system for automatic braking of a car. The braking force which is determined by the hydraulic fluid pressure is the dependent variable. There are two input variables defined for this system which are *speed* of the vehicle and the *distance* to the obstacle.  

There are many approaches for solving this problem. We want to use a **Rule-based** system using Fuzzy logic. The rules are described using colloquial language. 

### Importing libraries

In [13]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

### Universe definition
New Antecedent/Consequent objects hold universe variables and membership functions

In [14]:
speed = ctrl.Antecedent(np.arange(0, 121, 10), 'speed')
distance = ctrl.Antecedent(np.arange(0, 401, 10), 'distance')
pressure = ctrl.Consequent(np.arange(40, 201, 10), 'pressure')

### Membership functions
We can define the membership functions automatically using `automf()` method 

In [15]:
speed.automf(names = ['low', 'medium', 'high'])
distance.automf(names = ['near', 'average', 'far'])

Custom membership functions can be built interactively with a familiar, Pythonic API

In [16]:
pressure['low'] = fuzz.trimf(pressure.universe, [40, 40, 80])
pressure['below_average'] = fuzz.trimf(pressure.universe, [40, 80, 120])
pressure['average'] = fuzz.trimf(pressure.universe, [80, 120, 160])
pressure['above_average'] = fuzz.trimf(pressure.universe, [120, 160, 200])
pressure['high'] = fuzz.trimf(pressure.universe, [160, 200, 200])

Membership functions can be plotted using the `view()` method.

In [ ]:
# You can see how these look with .view()
speed['medium'].view()

In [ ]:
distance.view()

In [ ]:
pressure.view()

### Defining the rules
The heart of a rule-based system is the rule table, which defines the consequence (the output variable) in terms of the antecedents (the input variable(s)). We define rules that are shown in the table below:

|      |Near|Average|Far|
|---|:-:|:-:|:-:|
|Low|Average|Below Average|Low|
|Medium|Above Average|Average|Below Average|
|High|High|Above Average|Average|

In [20]:
rules = [
    ctrl.Rule(speed['high'] & distance['near'], pressure['high']),
    ctrl.Rule(
        (speed['high'] & distance['average']) |
        (speed['medium'] & distance['near']), 
        pressure['above_average']
    ),
    ctrl.Rule(
        (speed['high'] & distance['far']) | 
        (speed['medium'] & distance['average']) |
        (speed['low'] & distance['near']), 
        pressure['average']
    ),
    ctrl.Rule(
        (speed['medium'] & distance['far']) | 
        (speed['low'] & distance['average']), 
        pressure['below_average']
    ),
    ctrl.Rule(speed['low'] & distance['far'], pressure['low']),
]

The brake control system is defined using the rules described

In [21]:
brake_ctrl = ctrl.ControlSystem(rules)

And an instance of the system is created named `braking`

In [22]:
braking = ctrl.ControlSystemSimulation(brake_ctrl)

### Control system in action
Now we can pass the input values to the system and get the output value.

In [23]:
braking.input['speed'] = 85
braking.input['distance'] = 250

braking.compute()

We can get the output variable and plot the its fuzzy membership function

In [ ]:
print(braking.output['pressure'])
pressure.view(sim=braking)